# Intro

This note

In [17]:
# Imports
import pandas as pd
from faker import Faker

In [21]:
# Create a Faker object
fake = Faker(use_weighting=True, locale='en_US', include_private=False)

# Set the seed
Faker.seed(4242)

# Generate customer data
customers = []

for _ in range(10000):
    customers.append({
        'id': fake.uuid4(),
        'name': fake.first_name(),
        'last_name': fake.last_name(),
        'email': fake.email(),
        'registration_date': fake.date_between(start_date='-1y', end_date='today'),
        'birth_date': fake.date_of_birth(minimum_age=18, maximum_age=65),
        'street_address': fake.street_address(),
        'city': fake.city(),
        'state': fake.state(),
        'country': 'USA',
        'email_opt_in': fake.boolean(chance_of_getting_true=40)
        })
        
# Save the data to a DataFrame
customers_df = pd.DataFrame(customers)

# Export the csv file 
customers_df.to_csv('data/customers.csv', index=False)